In [58]:
#import our required modules
import pandas as pd
import zipfile
#import our custom module for etl functions
import etl_tools as etl

In [59]:
# set our variables for our source .zip files
beer_reviews_src = 'resources/beerreviews.zip'
us_breweries_src = 'resources/us-breweries.zip'

In [60]:
#unzip and import contents into dataframe
with zipfile.ZipFile(beer_reviews_src) as z:
   with z.open("beer_reviews.csv") as f:
      beer_reviews_src_df = pd.read_csv(f)

beer_reviews_src_df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [61]:
#review info about new dataframe
beer_reviews_src_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1586614 non-null int64
brewery_name          1586599 non-null object
review_time           1586614 non-null int64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586266 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


In [62]:
#get unique count of brewery_names
beer_reviews_src_df['brewery_name'].nunique()

5742

In [63]:
#get unique count of brewery_ids (this shows us the name need some cleaning!!!)
beer_reviews_src_df['brewery_id'].nunique()

5840

In [64]:
#unzip and import contents into dataframe
with zipfile.ZipFile(us_breweries_src) as z:
   with z.open("breweries_us.csv") as f:
      us_breweries_src_df = pd.read_csv(f)
us_breweries_src_df.head()

,brewery_name,type,address,website,state,state_breweries
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284


In [65]:
#review info of new dataframe
us_breweries_src_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2407 entries, 0 to 2406
Data columns (total 6 columns):
brewery_name       2407 non-null object
type               2407 non-null object
address            2407 non-null object
website            2399 non-null object
state              2407 non-null object
state_breweries    2407 non-null int64
dtypes: int64(1), object(5)
memory usage: 113.0+ KB


In [66]:
#get unique count of brewer_names
us_breweries_src_df['brewery_name'].nunique()

2308

<h4>clean up beer_reviews_src_df</h4>

In [67]:
#dropna to get rid of records with missing data
beer_reviews_src_df.dropna(inplace=True)
beer_reviews_src_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1518478 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1518478 non-null int64
brewery_name          1518478 non-null object
review_time           1518478 non-null int64
review_overall        1518478 non-null float64
review_aroma          1518478 non-null float64
review_appearance     1518478 non-null float64
review_profilename    1518478 non-null object
beer_style            1518478 non-null object
review_palate         1518478 non-null float64
review_taste          1518478 non-null float64
beer_name             1518478 non-null object
beer_abv              1518478 non-null float64
beer_beerid           1518478 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 162.2+ MB


In [68]:
#Build df of names and id's to normalize (it's )
brewery_names_df = beer_reviews_src_df[['brewery_id','brewery_name']].copy()
brewery_names_df.nunique()

brewery_id      5230
brewery_name    5155
dtype: int64

In [69]:
#drop duplicates
brewery_names_df.drop_duplicates('brewery_name',inplace=True)
brewery_names_df.set_index('brewery_id',inplace=True)
brewery_names_df.nunique()

brewery_name    5155
dtype: int64

In [70]:
#clean names
brewery_names_df = etl.clean_co_names(brewery_names_df,'brewery_name')
brewery_names_df

>Set Upper
>Remove commas
>Remove hyphens
>Remove text between parens
>replace AND with &
>Remove leading/trailing spaces
>Cleanco Pass1
>Remove dots
>Encode utf-8
>Cleanco Pass2


,brewery_name,clean_co
brewery_id,,
10325,Vecchio Birraio,b'VECCHIO BIRRAIO'
1075,Caldera Brewing Company,b'CALDERA BREWING'
163,Amstel Brouwerij B. V.,b'AMSTEL BROUWERIJ B V'
1454,Broad Ripple Brew Pub,b'BROAD RIPPLE BREW PUB'
850,Moon River Brewing Company,b'MOON RIVER BREWING'
...,...,...
21727,Gattopardo Cervejaria,b'GATTOPARDO CERVEJARIA'
6785,Brauerei Lasser GmbH,b'BRAUEREI LASSER'
12566,Wissey Valley Brewery,b'WISSEY VALLEY BREWERY'


In [71]:
brewery_names_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5155 entries, 10325 to 6102
Data columns (total 2 columns):
brewery_name    5155 non-null object
clean_co        5155 non-null object
dtypes: object(2)
memory usage: 120.8+ KB


In [72]:
#drop duplicates after cleanup
brewery_names_df.drop_duplicates('cleaned',inplace=True)
brewery_names_df

KeyError: Index(['cleaned'], dtype='object')

In [ ]:
#verify no more duplicates based on id
brewery_names_df.nunique()

In [ ]:
#join our names back in to beer_revieww_src_df
beer_reviews_src_df = beer_reviews_src_df.merge(brewery_names_df['cleaned'],left_on='brewery_id',right_index=True)
beer_reviews_src_df.head()

In [73]:
#clean our names from us_breweries_src_df
us_breweries_src_df = etl.clean_co_names(us_breweries_src_df,'brewery_name').set_index('cleaned',drop=True)
us_breweries_src_df

>Set Upper
>Remove commas
>Remove hyphens
>Remove text between parens
>replace AND with &
>Remove leading/trailing spaces
>Cleanco Pass1
>Remove dots
>Encode utf-8
>Cleanco Pass2


KeyError: "None of ['cleaned'] are in the columns"

In [ ]:
#count unique brewery_names
us_breweries_src_df['cleaned'].nunique()

In [ ]:
#inspect types
us_breweries_src_df['type'].unique()

In [ ]:
#determine closed locations
us_breweries_src_df['closed'] = us_breweries_src_df['type'].str.contains('Closed')

In [ ]:
#strip -Closed from type
us_breweries_src_df['type'] = us_breweries_src_df['type'].str.replace('-Closed','')

In [ ]:
#sort by open so that when we drop duplicates we keep the open location
us_breweries_src_df.sort_values('closed',ascending=False)

In [ ]:
beers = beer_reviews_src_df[['beer_beerid','beer_name','beer_style','beer_abv','cleaned']]
beers

In [175]:
reviews = beer_reviews_src_df[['review_profilename','review_time','review_overall','review_aroma','review_appearance','review_palate','review_taste','beer_beerid','cleaned']]
reviews

,review_profilename,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_beerid,cleaned
0,stcules,1234817823,1.5,2.0,2.5,1.5,1.5,47986,b'VECCHIO BIRRAIO'
1,stcules,1235915097,3.0,2.5,3.0,3.0,3.0,48213,b'VECCHIO BIRRAIO'
2,stcules,1235916604,3.0,2.5,3.0,3.0,3.0,48215,b'VECCHIO BIRRAIO'
3,stcules,1234725145,3.0,3.0,3.5,2.5,3.0,47969,b'VECCHIO BIRRAIO'
4,johnmichaelsen,1293735206,4.0,4.5,4.0,4.0,4.5,64883,b'CALDERA BREWING'
...,...,...,...,...,...,...,...,...,...
1586336,Kulrak,1164534578,4.0,4.0,2.5,3.5,4.5,13575,b'OUTBACK BREWERY'
1586337,vancurly,1156226344,2.0,2.5,2.5,2.0,2.0,13575,b'OUTBACK BREWERY'
1586338,rastaman,1066309446,1.0,1.0,1.0,1.0,1.0,13575,b'OUTBACK BREWERY'
1586339,rtepiak,1297373426,3.5,4.0,3.5,3.5,4.0,66101,b'GEORG MEINEL BIERBRAUEREI'


In [81]:
#join our cleaned dataframes together on the new cleaned name column
combined_beer = beer_reviews_src_df.merge(us_breweries_src_df,left_on='cleaned', index_right=True)
combined_beer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 801597 entries, 0 to 801596
Data columns (total 20 columns):
brewery_id            801597 non-null int64
brewery_name_x        801597 non-null object
review_time           801597 non-null int64
review_overall        801597 non-null float64
review_aroma          801597 non-null float64
review_appearance     801597 non-null float64
review_profilename    801597 non-null object
beer_style            801597 non-null object
review_palate         801597 non-null float64
review_taste          801597 non-null float64
beer_name             801597 non-null object
beer_abv              801597 non-null float64
beer_beerid           801597 non-null int64
cleaned               801597 non-null object
brewery_name_y        801597 non-null object
type                  801597 non-null object
address               801597 non-null object
website               801533 non-null object
state                 801597 non-null object
state_breweries       801597 no

In [133]:
combined_beer.head()

,brewery_id,brewery_name_x,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,...,beer_beerid,cleaned,brewery_name_y,type,address,website,state,state_breweries,city,zip
0,1075,Caldera Brewing Company,2010-12-30 18:53:26,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,...,64883,b'CALDERA BREWING',Caldera Brewing Company,Microbrewery,540 Clover Ln.,http://www.calderabrewing.com/,Oregon,156,Ashland,97520
1,1075,Caldera Brewing Company,2012-01-02 17:17:39,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,...,52159,b'CALDERA BREWING',Caldera Brewing Company,Microbrewery,540 Clover Ln.,http://www.calderabrewing.com/,Oregon,156,Ashland,97520
2,1075,Caldera Brewing Company,2011-10-19 02:25:15,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,...,52159,b'CALDERA BREWING',Caldera Brewing Company,Microbrewery,540 Clover Ln.,http://www.calderabrewing.com/,Oregon,156,Ashland,97520
3,1075,Caldera Brewing Company,2011-05-24 22:26:58,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,...,52159,b'CALDERA BREWING',Caldera Brewing Company,Microbrewery,540 Clover Ln.,http://www.calderabrewing.com/,Oregon,156,Ashland,97520
4,1075,Caldera Brewing Company,2010-11-22 19:35:03,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,...,52159,b'CALDERA BREWING',Caldera Brewing Company,Microbrewery,540 Clover Ln.,http://www.calderabrewing.com/,Oregon,156,Ashland,97520


In [85]:
#split out address, overwrite original address and state fields with newly parsed data from address
combined_beer[['address','city','state','zip']] = combined_beer['address'].str.rsplit(',',n=3,expand=True)
combined_beer.head(2)

ValueError: Columns must be same length as key

In [87]:
#convert from epoch time to datetime
combined_beer['review_time'] = pd.to_datetime(combined_beer['review_time'],unit='s')
combined_beer.head()

,brewery_id,brewery_name_x,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,...,beer_beerid,cleaned,brewery_name_y,type,address,website,state,state_breweries,city,zip
0,1075,Caldera Brewing Company,2010-12-30 18:53:26,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,...,64883,b'CALDERA BREWING',Caldera Brewing Company,Microbrewery,540 Clover Ln.,http://www.calderabrewing.com/,Oregon,156,Ashland,97520
1,1075,Caldera Brewing Company,2012-01-02 17:17:39,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,...,52159,b'CALDERA BREWING',Caldera Brewing Company,Microbrewery,540 Clover Ln.,http://www.calderabrewing.com/,Oregon,156,Ashland,97520
2,1075,Caldera Brewing Company,2011-10-19 02:25:15,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,...,52159,b'CALDERA BREWING',Caldera Brewing Company,Microbrewery,540 Clover Ln.,http://www.calderabrewing.com/,Oregon,156,Ashland,97520
3,1075,Caldera Brewing Company,2011-05-24 22:26:58,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,...,52159,b'CALDERA BREWING',Caldera Brewing Company,Microbrewery,540 Clover Ln.,http://www.calderabrewing.com/,Oregon,156,Ashland,97520
4,1075,Caldera Brewing Company,2010-11-22 19:35:03,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,...,52159,b'CALDERA BREWING',Caldera Brewing Company,Microbrewery,540 Clover Ln.,http://www.calderabrewing.com/,Oregon,156,Ashland,97520


In [ ]:
#break out our data into proper dimensions
breweries = combined_beer[['brewery_id','brewery_name','address','city','state','zip','website','state_breweries']]

In [34]:
#count of beerid
#count of beer_style
#count of review_profilenames
#avg of review_overall
#avg of review_aroma
#avg of review_appearance
#avg of review_palate
#avg of review_taste


In [35]:
#export to sqlite
import sqlite3 as sq
table_list = []
table_list.append(pd.DataFrame(combined_beer))
dbname = 'beer_data'

sql_data = f'data/{dbname}.db'
#etl.sq.register_adapter(etl.np.int64, lambda val: float(val))
conn = sq.connect(sql_data)
cur = conn.cursor()

try:
    dropstring = '''drop table if exists "{0}"'''.format('combined_beer')
    print(dropstring)
    cur.execute(dropstring)
    combined_beer.to_sql('combined_beer',conn, if_exists='replace', index=False)
except:
    pass
conn.commit()
conn.close() 

drop table if exists "combined_beer"


In [36]:
#export to parquet
combined_beer.reset_index(drop=True).to_parquet('data/combined_beer.parquet', engine='fastparquet', compression='gzip')

In [145]:
grouped = us_breweries_src_df.groupby('cleaned')[['brewery_name','address']]

In [ ]:
grouped[['address','city','state','zip']] = combined_beer['address'].str.rsplit(',',n=3,expand=True)
combined_beer.head(2)

In [165]:
us_breweries = us_breweries_src_df.copy()

In [171]:
#test = {k: g.to_dict(orient='series') for k,g in us_breweries.groupby("cleaned")}
#test


In [172]:
#grouped = us_breweries_src_df.to_dict(orient='series')
#grouped

#d = {}
#for i in us_breweries_src_df['cleaned'].unique():
#    d[i] = [{df['value1'][j]: df['value2'][j]} for j in df[df['name']==i].index]
    

In [143]:
us_breweries_src_df

,brewery_name,type,address,website,state,state_breweries,closed
cleaned,,,,,,,
b'VALLEY BREWING',Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284,False
b'VALLEY BREWING',Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284,False
b'VALLEY BREWING',Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284,False
b'UKIAH BREWING',Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284,False
b'TUSTIN BREWING',Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284,False
...,...,...,...,...,...,...,...
b'CAPITOL CITY BREWING',Capitol City Brewing Company (Capitol Hill),Brewpub,"2 Massachusetts Ave., Washington, DC, Washingt...",http://www.capcitybrew.com/,washington-dc,9,False
b'CAPITOL CITY BREWING',Capitol City Brewing Company,Brewpub,"1100 New York Ave. NW, Washington, DC, Washing...",http://www.capcitybrew.com/,washington-dc,9,False
b'OLD HARBOR BREWERY',Old Harbor Brewery,Brewpub,"Gen del Valle 1021 Urb Delicias, San Juan, PR,...",http://oldharborbrewery.com/,puerto-rico,3,False
